In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Um einen dependency error zu vermeiden, wird transformers 4.28.0 installiert

In [1]:
!pip install datasets 
!pip install transformers==4.28.0
!pip install --upgrade accelerate
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 88.0 MB/s eta 0:00

# 0. Preprocessing

In [2]:
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, AutoModel
import evaluate
import numpy as np

Download des Datasets:

In [3]:
emotion_dataset = load_dataset("tweet_eval", "emotion")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emotion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Kurzes Betrachten des Datasatzes im Ganzen...:

In [4]:
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3257
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1421
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 374
    })
})

...und eines Beispieleintrags:

In [5]:
emotion_dataset['train'][0]

{'text': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 'label': 2}

Als Checkpoint verwende ich "bert-base-uncased", weil es sich als großes Modell für verschiedenste Tasks eignet und noch nicht auf dem tweet_eval Datensatz finegetuned wurde.

In [6]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Die tokenize_function dient der Tokenisierung des Datensatzes und entspricht der Form, die in der Übung gezeigt wurde.

In [7]:
def tokenize_function(sample):
  return tokenizer(sample["text"], truncation=True)

In [8]:
tokenized_datasets = emotion_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3257 [00:00<?, ? examples/s]

Map:   0%|          | 0/1421 [00:00<?, ? examples/s]

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

Auch der data_collator wird analog zur Übung instanziiert.

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#1. Training/ 2. Evaluation

Wie aus https://huggingface.co/datasets/tweet_eval zu entnehmen, gibt es im emotion subset 4 relevante label, was hier als Parameter übergeben wird

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Für die Evaluation verwende ich die evaluate library

Die einzelnen, für die Text-Klassifikation geeigneten Metriken (vgl. https://huggingface.co/tasks/text-classification) werden geladen

In [11]:
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")
accuracy_metric= evaluate.load("accuracy")

Entsprechend wird auch hier die compute_metrics Funktion mit den gerade geladenen Metriken gefüllt

In [16]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="macro"))
    results.update(precision_metric.compute(predictions=preds, references = labels, average="macro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="macro"))
    results.update(accuracy_metric.compute(predictions=preds, references = labels))
    return results

Die Trainingsparameter sind der Übung angelehnt

In [13]:
batch_size = 8
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = f"{checkpoint}-finetuned-twitter-emotion"
training_args = TrainingArguments(output_dir = model_name, 
                                  num_train_epochs = 2, 
                                  learning_rate = 2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  evaluation_strategy="epoch", 
                                  disable_tqdm = False, 
                                  logging_steps = logging_steps,
                                  log_level="error")

Auch der Trainer wird analog zur Übung geladen, mit dem Unterschied, dass die kompletten Train- und Test-Splits verwendet werden.

In [17]:
trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    # da der Datensatz nicht all zu groß ist, können wir das Model auf allen Einträgen finetunen
    train_dataset=tokenized_datasets["train"], 
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [18]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy
1,0.309500,0.761979,0.778696,0.793738,0.767834,0.816327
2,0.219600,0.792893,0.776037,0.777984,0.774369,0.812104


TrainOutput(global_step=816, training_loss=0.26394818432848244, metrics={'train_runtime': 85.2412, 'train_samples_per_second': 76.418, 'train_steps_per_second': 9.573, 'total_flos': 130324020930792.0, 'train_loss': 0.26394818432848244, 'epoch': 2.0})

Der 

In [19]:
trainer.evaluate(eval_dataset=tokenized_datasets['test'])

{'eval_loss': 0.7928932309150696,
 'eval_f1': 0.7760370521594682,
 'eval_precision': 0.7779843022121412,
 'eval_recall': 0.7743687605646147,
 'eval_accuracy': 0.8121041520056298,
 'eval_runtime': 3.6359,
 'eval_samples_per_second': 390.826,
 'eval_steps_per_second': 48.956,
 'epoch': 2.0}

Hier speichere ich mein Modell

In [ ]:
model_path = '/content/drive/MyDrive/Sommersemester_2023/NLP/SL1_Model'
trainer.save_model(model_name, model_path)

# 3. Vergleich

Zum Vergleich verwende das Modell 'apetulante/bert-emotion' https://huggingface.co/apetulante/bert-emotion, das auf distilbert-base-cased basiert, weil mich die Performanz zwischen bert und distilbert basierten Modellen interessiert.

In [20]:
comparison_model = AutoModelForSequenceClassification.from_pretrained('apetulante/bert-emotion')
comparison_tokenizer = AutoTokenizer.from_pretrained('apetulante/bert-emotion')

evaluate benötigt einen evaluator, der mit der gewünschten task instantiiert werden muss

In [21]:
from evaluate import evaluator

task_evaluator = evaluator("text-classification")

Idealerweise würde ich hier alle Metriken, die auch oben verwendet wurden noch einmal für dieses Modell evaluieren, was aber aufgrund von einer Fehlermeldung bei evaluate.combine bei mir nicht funktioniert hat (laut Dokumentation aber eigentlich funktionieren sollte: https://huggingface.co/docs/evaluate/base_evaluator)

Die erwähnten Fehler bei evaluate.combine werden auch hier von anderen Nutzern aufgeführt: https://discuss.huggingface.co/t/combining-metrics-for-multiclass-predictions-evaluations/21792/10

In [22]:
eval_results = task_evaluator.compute(
    model_or_pipeline=comparison_model,
    tokenizer=comparison_tokenizer,
    #metric=evaluate.load("f1"), #theoretisch wären auch mehrere Metriken hier möglich mit evaluate.combine(["accuracy", "recall", "precision", "f1"])
    #beim nochmaligem Durchlauf hat auch diese Zeile dieselbe Fehlermeldung produziert, weswegen ich auf den default-value des evaluators (accuracy) zurückgreifen muss
    data=tokenized_datasets['test'],
    label_mapping={"anger": 0, "joy": 1, "optimism": 2, "sadness": 3}
)

print(eval_results)

{'accuracy': 0.7860661505981703, 'total_time_in_seconds': 11.16991482200001, 'samples_per_second': 127.21672659501671, 'latency_in_seconds': 0.007860601563687552}


Bei Vergleich der F1-Metriken (0.78 vs 0.74) lässt sich feststellen, dass mein Modell tatsächlich einen besseren Score erzielen konnte. Mutmaßlich lässt sich dies darauf zurückführen, dass mein Modell auf bert-base-uncased trainiert wurde, während das Vergleichsmodell auf distilbert-base-cased finegetuned wurde, also einer destillierten Version von BERT. Auch wurde das Vergleichsmodell mit anderen Hyperparametern trainiert, so etwa einer geringeren batch_size, einer höheren learning_rate und 3 statt 2 Epochen - Parameter, die sich natürlich allesamt auf die Performanz des finalen Modells auswirken können. Notieren muss man aber natürlich, dass mein Modell mutmaßlich auch etwas overfitted sein könnte, da wir hier zum evaluieren ja den test-split (der auch beim Training verwendet wurde) und nicht den validate-split verwenden.  

Notiz: Wie oben erwähnt, ging beim zweiten Durchlauf nur noch die Accuracy-Metrik, wo mein Modell aber mit 0.812 gegenüber 0.786 auch besser performt.